In [1]:
#Auxiliares
import os
import pickle
from time import time
from datetime import datetime
from json import dumps

#Dados
import pandas as pd

#preprocessing and transformation
from sklearn.preprocessing import StandardScaler,MinMaxScaler,MaxAbsScaler

#Machine learning
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

#Metrics
from sklearn.metrics import classification_report

seed = 42

In [2]:
#Variables
base_path = 'D:/03. Documentos/Mestrado/22032020 - Experimentos/05. Organizado/03. Datasets/Clareza-Unbalanced-Binary-COH-METRIX'
save_path = 'output'

x_train_file = 'X_train.csv'
y_train_file = 'y_train.csv'
x_test_file = 'X_test.csv'
y_test_file = 'y_test.csv'

label='Clareza'

#Load data
X_train = pd.read_csv(os.path.join(base_path, x_train_file), sep=';', encoding='utf-8', decimal='.')
y_train = pd.read_csv(os.path.join(base_path, y_train_file), sep=';', encoding='utf-8', decimal='.')
X_test = pd.read_csv(os.path.join(base_path, x_test_file), sep=';', encoding='utf-8', decimal='.')
y_test = pd.read_csv(os.path.join(base_path, y_test_file), sep=';', encoding='utf-8', decimal='.')

#Checking on data
print(X_train.columns)
print(X_train.shape)
print(y_train[label].value_counts())
print(y_test[label].value_counts())

Index(['pid', 'req-text', 'resp-text', '1funct-request', '2pronoun-request',
       '3ppron-request', '4i-request', '5we-request', '6you-request',
       '7shehe-request',
       ...
       'sentence_length_min', 'sentence_length_standard_deviation',
       'short_sentence_ratio', 'std_noun_phrase', 'verb_diversity',
       'verbs_max', 'verbs_min', 'verbs_standard_deviation',
       'long_sentence_ratio', 'tempo_resposta'],
      dtype='object', length=310)
(4368, 310)
2    2736
0    1632
Name: Clareza, dtype: int64
2    1204
0     669
Name: Clareza, dtype: int64


In [3]:
#type(X_train.adjective_ratio.iloc[0])
#type(X_train.flesch.iloc[0])
#X_train.to_clipboard(excel=True)

In [25]:
# Define pipeline and GridSearch CV

params = {
    'scaler': MaxAbsScaler(),
    'clf_solver':'lbfgs',
    'clf_max_iter': 10000,
    #'clf_penalty': 'l1',
    'gs_cv': 10,    
    'gs_scoring': 'f1_macro'
}

pipelineWord = Pipeline([    
    #('scaler', MaxAbsScaler()),
    ('scaler', params.get('scaler')),    
    #('scaler', MinMaxScaler()),
    ('clf', LogisticRegression(random_state=seed,
                               n_jobs=6,
                               class_weight='balanced',
                               #penalty=params.get('clf_penalty'),
                               solver=params.get('clf_solver'),
                               max_iter=params.get('clf_max_iter')))
])

gs_parameters = {    
    'clf__C': (1e-2, 0.1, 1, 10, 100, 1000)
}

# Define grid search
grid_search_word = GridSearchCV(pipelineWord,
                               gs_parameters,
                               cv=params.get('gs_cv'),
                               scoring=params.get('gs_scoring'),
                               n_jobs=6,
                               verbose=10
                               )

In [26]:
X = X_train.iloc[:, 3+128:-1]
Y = y_train[label]
x = X_test.iloc[:, 3+128:-1]
y = y_test[label]

In [27]:
print("Executando Gridsearch para Respostas - Binary COH-METRIX - Classe Clareza")

now = str(datetime.now()).split('.')[0].replace('-', '_').replace(' ', '_').replace(':', '_')
print(now)

t0 = time()
grid_search_word.fit(X, Y)
print("done in %0.3fs" % (time() - t0))
print("Best score: %0.3f" % grid_search_word.best_score_)
print("Best parameters set:")
best_parameters = grid_search_word.best_estimator_.get_params()
for param_name in sorted(gs_parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Executando Gridsearch para Respostas - Binary COH-METRIX - Classe Clareza
2020_06_10_19_37_39
Fitting 10 folds for each of 6 candidates, totalling 60 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 tasks      | elapsed:    0.1s
[Parallel(n_jobs=6)]: Batch computation too fast (0.1701s.) Setting batch_size=2.
[Parallel(n_jobs=6)]: Done   6 tasks      | elapsed:    0.1s
[Parallel(n_jobs=6)]: Done  14 tasks      | elapsed:    0.9s
[Parallel(n_jobs=6)]: Done  28 tasks      | elapsed:    3.0s
[Parallel(n_jobs=6)]: Batch computation too slow (2.6839s.) Setting batch_size=1.
[Parallel(n_jobs=6)]: Done  42 out of  60 | elapsed:    9.7s remaining:    4.1s
[Parallel(n_jobs=6)]: Done  56 out of  60 | elapsed:   22.2s remaining:    1.5s
[Parallel(n_jobs=6)]: Done  60 out of  60 | elapsed:   22.7s finished


done in 37.430s
Best score: 0.557
Best parameters set:
	clf__C: 100


In [30]:
# Saving Model
f_save = now + '_Clareza_Resposta_Binary_Unbalanced_coh_metrix.sav'
pickle.dump(grid_search_word, open(os.path.join(save_path, f_save),'wb'))

# Saving Parameters
with open(os.path.join(save_path, 'params.txt'),'a') as f:
    f.write('\n\n' + ('#'*60))
    f.write('\n'+f_save + '\n\n')
    f.write('Parameters:\n')
    f.write(dumps(str(params)) + '\n')
    f.write('\nGridSearch Best Parameters:\n')
    for param_name in sorted(gs_parameters.keys()):        
        f.write("%s: %r" % (param_name, best_parameters[param_name]) + '\n')    

# Validation

In [28]:
# Predictions
y_pred_train = grid_search_word.predict(X)
y_pred_test = grid_search_word.predict(x)

In [29]:
print('#'*50)
print('Report for TRAIN')
print('#'*50)
print(classification_report(Y, y_pred_train))

print('#'*50)
print('Report for TEST')
print('#'*50)
print(classification_report(y, y_pred_test))

##################################################
Report for TRAIN
##################################################
              precision    recall  f1-score   support

           0       0.49      0.64      0.55      1632
           2       0.73      0.60      0.66      2736

    accuracy                           0.61      4368
   macro avg       0.61      0.62      0.61      4368
weighted avg       0.64      0.61      0.62      4368

##################################################
Report for TEST
##################################################
              precision    recall  f1-score   support

           0       0.45      0.59      0.51       669
           2       0.72      0.59      0.65      1204

    accuracy                           0.59      1873
   macro avg       0.58      0.59      0.58      1873
weighted avg       0.62      0.59      0.60      1873



In [31]:
with open(os.path.join(save_path, 'params.txt'),'a') as f:
    f.write('\n\n' + ('#'*60))
    f.write('\nReport for TRAIN')
    f.write('\n' + ('#'*60))
    f.write('\n' + classification_report(Y, y_pred_train))
    
    f.write('\n\n' + ('#'*60))
    f.write('\nReport for TEST')
    f.write('\n' + ('#'*60))
    f.write('\n' + classification_report(y, y_pred_test))